# Kimberlite TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a synthetic kimberlitic model.

In [116]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [117]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [118]:
model_dir = 'data/model.pickle'
grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'
data_dir = 'data/flightlines_grid_2100pts_100k/data.pickle'

In [119]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [120]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [121]:
with open(data_dir) as w:
        data = pickle.load(w)

In [122]:
# observed data and observation points
dobs = data['amf_obs']
xp = grid['x']
yp = grid['y']
zp = grid['z']
N = grid['N']

### Parameters of the initial model

In [123]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']# - 5.
decs = model['dec']# - 5.

# depth to the top, thickness and radius
z0 = model['z0']# - 50.
dz = 350.
r = 2000.

# total magnetization
intensity = model['intensity']# + 1.
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = data['main_field']

# predict data
d0 = np.sqrt(polyprism.bx(xp, yp, zp, model0)**2. +
             polyprism.by(xp, yp, zp, model0)**2. +
             polyprism.bz(xp, yp, zp, model0)**2.)

In [124]:
xp.size

2100

### Limits

In [125]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 4000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [126]:
# variation for derivatives
deltax = 0.01*np.max(1000)
deltay = 0.01*np.max(1000)
deltar = 0.01*np.max(1000)
deltaz = 0.01*np.max(1000)

### Outcropping parameters

In [127]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [128]:
if grid_dir[9:20] == 'flightlines':
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'amf_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [129]:
# output of inversion
inversion = dict()

### Regularization parameters

In [130]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-3  # adjacent radial distances within each prism
a2 = 1.0e-3   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4     # vertically adjacent origins
a6 = 1.0e-8   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-6     # zero order Tikhonov on thickness of each prism

In [131]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [132]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [133]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [134]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [data['main_field'][0], data['main_field'][1]]

### Inversion

In [135]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_amf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+01   init obj.: 2.24985e+05  fin obj.: 1.21262e+05
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.21262e+05  fin obj.: 4.09726e+04
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 4.09726e+04  fin obj.: 3.88981e+03
it:  3   it_marq:  0   lambda: 1e-02   init obj.: 3.88981e+03  fin obj.: 3.76436e+03
it:  4   it_marq:  0   lambda: 1e-03   init obj.: 3.76436e+03  fin obj.: 2.31136e+03
it:  5   it_marq:  0   lambda: 1e-04   init obj.: 2.31136e+03  fin obj.: 4.34007e+04
it:  5   it_marq:  1   lambda: 1e-03   init obj.: 2.31136e+03  fin obj.: 3.90374e+04
it:  5   it_marq:  2   lambda: 1e-02   init obj.: 2.31136e+03  fin obj.: 1.67894e+04
it:  5   it_marq:  3   lambda: 1e-01   init obj.: 2.31136e+03  fin obj.: 7.17325e+02
it:  6   it_marq:  0   lambda: 1e-02   init obj.: 7.17325e+02  fin obj.: 3.90633e+02
it:  7   it_marq:  0   lambda: 1e-03   init obj.: 3.90633e+02  fin obj.: 1.06460e+03
it:  7   it_marq:  1   lambda: 1e-02   init obj.: 3.90633e+02  fi

# Results

In [136]:
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

In [137]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)